# **Aula 13 - DDS4 - Machine Learning com PyCaret**

## **DADOS DE CUSTOS MÉDICOS**

In [ ]:
#instalar pycaret
!pip install pycaret[full]

In [2]:
#Criando drive no colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Validando pasta que origem
import os
os.chdir("/content/drive/My Drive")
!ls

 Apostilas
'Aula 13 - DDS4 - Machine Learning com PyCaret.ipynb'
 Aula_3_1_Nocoes_de_Inferencia_Simulacao_de_uma_Pesquisa_Eleitoral.ipynb
 insurance.csv


In [ ]:
from pycaret.datasets import get_data
dados = get_data('/content/drive/My Drive/insurance') 

In [ ]:
dados.head()

In [ ]:
import seaborn as sns
sns.scatterplot(x = 'bmi', y = 'charges', hue = 'smoker', data = dados);

---

## **Objetivo Primário: construir um bom modelo para previsão dos custos médicos de uma pessoa**

In [6]:
import pycaret.regression as pyreg 

In [7]:
?pyreg.setup

In [ ]:
# setup
regressao = pyreg.setup(data = dados, target = 'charges', 
                       combine_rare_levels = True,
                       normalize = True)

In [9]:
pyreg.models().shape

(25, 3)

In [ ]:
rf = pyreg.create_model('rf')

In [ ]:
rf_tunado = pyreg.tune_model(rf, optimize = 'RMSLE')

In [ ]:
# Comparando os modelos implementados, ordenando pelo RMSLE
melhores = pyreg.compare_models(sort = 'RMSLE', n_select = 15)

In [ ]:
melhores

In [ ]:
melhores_tunados = []

for i in range(len(melhores)):
    melhores_tunados.append(pyreg.tune_model(melhores[i], optimize = 'RMSLE'))

In [ ]:
for i in range(len(melhores_tunados)):
    ccc = pyreg.predict_model(melhores_tunados[i]) 

In [ ]:
combinado9 = pyreg.blend_models(melhores_tunados[:9])

In [ ]:
et = pyreg.create_model('et')

In [ ]:
et_tunado = pyreg.tune_model(et, optimize = 'RMSLE')

In [ ]:
# Performance nos dados de validacao
pred = pyreg.predict_model(et_tunado)
pred

In [ ]:
pred['Label']

In [ ]:
pyreg.evaluate_model(et_tunado)


In [23]:
# Utilizando o modelo escolhido nos dados todos para gerar o modelo final
modelo_final = pyreg.finalize_model(et_tunado)

In [26]:
# Salvando o modelo
pyreg.save_model(modelo_final, 'meu-modelo-para-charges')

Transformation Pipeline and Model Succesfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=[], target='charges',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strategy...
                  ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0,
                                      criterion='mae', max_depth=3,
                                      max_features=1.0, max_leaf_nodes=None,
                                      max_samples=None,
                                      min_

---

In [27]:
# Deploy Básico
import pandas as pd

def CustoSeguro(age, sex, bmi, children, smoker, region):
    dados0 = {'age': [age], 'sex': [sex], 'bmi': [bmi], 'children': [children], 'smoker': [smoker], 'region': [region]}
    dados = pd.DataFrame(dados0)

    modelo = pyreg.load_model('meu-modelo-para-charges')

    pred = float(pyreg.predict_model(modelo, data = dados)['Label'].round(2))
    valor = round(1.8*pred, 2)
    
    print(' ')
    print('Idade: {}'.format(age))
    print('Sexo: {}'.format(sex))
    print('IMC: {}'.format(bmi))
    print('Crianças: {}'.format(children))
    print('Fumante: {}'.format(smoker))
    print('Região: {}'.format(region))
    print(' ')
    print('Custo do Seguro: ${}'.format(pred))
    print('Valor de Venda: ${}'.format(valor))


In [ ]:
CustoSeguro(age = 34, sex = 'male', bmi = 30, children = 0, smoker =  'yes', region = 'southwest')

In [ ]:
CustoSeguro(45, 'female', 20, 2, 'yes', 'southwest')

In [ ]:
CustoSeguro(age = 45, 
            sex = 'female', 
            bmi = 40, 
            children = 5, 
            smoker =  'yes', 
            region = 'southwest')

---

## **Objetivo secundário: construir um modelo para detectar fumantes e possíveis fraudadores dessa variável.**

In [31]:
from pycaret.classification import *

In [ ]:
class_setup = setup(data = dados,
                    target = 'smoker',
                    normalize = True, 
                    ignore_low_variance = True,
                    remove_multicollinearity = True, 
                    multicollinearity_threshold = 0.95,
                    n_jobs = -1)

In [ ]:
models()

In [ ]:
todos = compare_models(n_select = 18, turbo = True)

In [ ]:
modelo = create_model('catboost')

In [ ]:
modelo_tunado = tune_model(modelo, optimize = 'Accuracy')

In [ ]:
top7_tunados = [tune_model(i) for i in todos[:7]]


In [ ]:
top7_tunados

In [39]:
import pandas as pd
results = pd.DataFrame()

In [ ]:
for i in range(len(top7_tunados)):
    ccc = predict_model(top7_tunados[i]) 

In [ ]:
combinacao_soft = blend_models(top7_tunados, method = 'soft') # os sete melhores modelos soltam probabilidades, então posso fazer combinação 'soft'

In [ ]:
combinacao_hard = blend_models(top7_tunados, method = 'hard')

Tanto as combinações 'hard' quanto 'soft' tiveram performances superiores a qualquer modelo individual. Por tanto, para modelo de produção, escolheremos a combinação 'soft', pois a 'hard' não solta probabilidades da variável resposta. 

In [43]:
preds = predict_model(combinacao_soft, data = dados)

In [ ]:
preds.sample(20).sort_values(by = 'Score')

In [ ]:
plot_model(combinacao_soft)

In [ ]:
plot_model(combinacao_soft, plot = 'confusion_matrix')

---

In [47]:
pred = predict_model(combinacao_soft)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.9726,0.9945,0.9589,0.8974,0.9272,0.9103,0.9111


## Utilizando a função predict_proba podemos tomar os valores das probabilidades dada pelo modelo. E podemos ver que o que acontece na variável 'Score' é que a probabilidade apresentada é sempre a de pertencer a resposta classificada, por isso então a suposta incoerência nos valores observados. 

---

## Utilizando o modelo selecionado na propria base para identificar possiveis fraudadores

In [ ]:
pred = predict_model(combinacao_soft, data = dados)
pred

## Queremos selecionar aqueles em que a resposta observada é não fumante, mas que nosso modelo indica que é fumante. Vamos ordenar o resultado pelo Score, pois quanto mais alto, maior a probabilidade ser fumante e, portanto, maior o risco de ser um fraudador. 

In [ ]:
possiveis_fraudes = pred[(pred['Label'] == 'yes') & (pred['smoker'] == 'no')].sort_values(by = 'Score', ascending = False)
possiveis_fraudes

## Portanto, o conjunto de dados acima é uma lista dos possíveis fraudadores, ordenada por aqueles mais prováveis de serem de fato fraudadores. Uma instituição poderia, por exemplo, tem um patamar em que caso a probabilidade seja maior que esse patamar, então uma investigação seria realizada. Por exemplo, se esse patamar fosse 70%, os individuos a serem investigados então seriam os seguintes:

In [ ]:
investigar = possiveis_fraudes[possiveis_fraudes['Score'] > 0.7]
investigar

In [51]:
modelo_final = finalize_model(combinacao_soft)

In [ ]:
# Salvando o modelo
save_model(modelo_final, 'meu-modelo-para-smoker')

In [53]:
# Deploy Básico

def DetectorDeFumante(age, sex, bmi, children, region, charges):
    dados0 = {'age': [age], 'sex': [sex], 'bmi': [bmi], 'children': [children], 'region': [region], 'charges': [charges]}
    dados = pd.DataFrame(dados0)

    modelo = load_model('meu-modelo-para-smoker')
 
    pred = predict_model(modelo, data = dados)
    resp = 'NÃO' if pred['Label'][0] == 'no' else 'SIM' 
    prob = pred['Score'][0] 


    print(' ')
    print('Idade: {}'.format(age))
    print('Sexo: {}'.format(sex))
    print('IMC: {}'.format(bmi))
    print('Crianças: {}'.format(children))
    print('Região: {}'.format(region))
    print('Custos: {}'.format(charges))
    print(' ')
    print('É fumante: {}, com probabilidade {:.2f}%'.format(resp, 100*prob)) 

In [57]:
DetectorDeFumante(age = 30, sex = 'male', bmi = 25, children = 0, region = 'southwest', charges = 2000)

Transformation Pipeline and Model Successfully Loaded
 
Idade: 30
Sexo: male
IMC: 25
Crianças: 0
Região: southwest
Custos: 2000
 
É fumante: NÃO, com probabilidade 88.75%


In [56]:
DetectorDeFumante(age = 30, 
                  sex = 'male', 
                  bmi = 25, 
                  children = 0, 
                  region = 'southwest', 
                  charges = 10000)

Transformation Pipeline and Model Successfully Loaded
 
Idade: 30
Sexo: male
IMC: 25
Crianças: 0
Região: southwest
Custos: 10000
 
É fumante: NÃO, com probabilidade 85.50%
